In [1]:
# test.py
import pandas as pd
from ArticleTools import crawling_news

df = pd.DataFrame(crawling_news.get_req())
df

,title,points,link
0,Forbes Marketplace - 숙주를 먹어치우려는 기생 SEO 회사의 진실 ...,4,https://larslofgren.com/forbes-marketplace/
1,"EU, 아이폰에 IOS 타사 개방 요구 (news.naver.com)",3,https://news.naver.com/mnews/article/029/00029...
2,"GN⁺: 닌텐도, Palworld 개발사 PocketPair 상대로 특허권 침해 소...",1,https://www.nintendo.co.jp/corporate/release/e...
3,GN⁺: Scramble - Grammarly 대체 오픈소스 (github.com...,7,https://github.com/zlwaterfield/scramble
4,"Oracle, 이제 JavaScript를 놓아줘야 할 때입니다. (javascrip...",14,https://javascript.tm/
5,Safari 18.0의 WebKit 기능들 (webkit.org),4,https://webkit.org/blog/15865/webkit-features-...
6,GN⁺: 책을 쓰지 말아야 하는 이유 (gwern.net),12,https://gwern.net/book-writing
7,GN⁺: MrBeast의 프로덕션 성공 비법 (유출된 PDF) (simonwilli...,34,https://simonwillison.net/2024/Sep/15/how-to-s...
8,The HTTP Query Method (ietf.org),11,https://www.ietf.org/archive/id/draft-ietf-htt...
9,"애플, iOS 18 출시 (apple.com)",4,https://www.apple.com/kr/newsroom/2024/09/ios-...


In [2]:
df['link']

0           https://larslofgren.com/forbes-marketplace/
1     https://news.naver.com/mnews/article/029/00029...
2     https://www.nintendo.co.jp/corporate/release/e...
3              https://github.com/zlwaterfield/scramble
4                                https://javascript.tm/
5     https://webkit.org/blog/15865/webkit-features-...
6                        https://gwern.net/book-writing
7     https://simonwillison.net/2024/Sep/15/how-to-s...
8     https://www.ietf.org/archive/id/draft-ietf-htt...
9     https://www.apple.com/kr/newsroom/2024/09/ios-...
10    https://peterszasz.com/how-to-lead-your-team-w...
11          https://www.youtube.com/watch?v=j2VBKvvmKKM
12     https://www.matthewball.co/all/fansprofitandloss
13    https://www.reuters.com/technology/how-intel-l...
14    https://rethinkingsoftware.substack.com/p/why-...
15    https://timculpan.substack.com/p/apple-mobile-...
16       https://www.swift.org/blog/announcing-swift-6/
17                  https://news.hada.io/topic?i

In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader(df['link'][5])
data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
all_splits = text_splitter.split_documents(data)

In [5]:
all_splits

[Document(metadata={'source': 'https://webkit.org/blog/15865/webkit-features-in-safari-18-0/', 'title': '  WebKit Features in Safari 18.0 | WebKit', 'language': 'No language found.'}, page_content='WebKit Features in Safari 18.0 | WebKit\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\nWebKit\n\nDownloads\nFeature Status\n\nCSS Features\nStandards Positions\n\n\nDocumentation\n\nWeb Inspector\nTracking Prevention\n\n\nPolicies\n\nProject Goals\nBug Prioritization\nBug Report Guidelines\nCode Style Guidelines\nCommit and Review Policy\nFeature Policy\nSecurity Policy\nTracking Prevention Policy\n\n\nContribute'),
 Document(metadata={'source': 'https://webkit.org/blog/15865/webkit-features-in-safari-18-0/', 'title': '  WebKit Features in Safari 18.0 | WebKit', 'language': 'No language found.'}, page_content='Getting Started\nContributing Code\nTesting Contributions\nHow to Report Bugs\nGitHub Repos

In [6]:
import requests
from bs4 import BeautifulSoup

contents = []

for i in range(len(df['link'])):
    res = requests.get(df['link'][i], auth=("user", "pass")) 

    if res.status_code == 200:        
        soup = BeautifulSoup(res.text, 'lxml')
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()  # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        contents.append(text)

    elif res.status_code == 403:
        contents.append('e403')

    else:
        contents.append('e404')

In [7]:
pd.concat([df, pd.Series(contents, name='content')], axis=1)

,title,points,link,content
0,Forbes Marketplace - 숙주를 먹어치우려는 기생 SEO 회사의 진실 ...,4,https://larslofgren.com/forbes-marketplace/,Forbes Marketplace: The Parasite SEO Company T...
1,"EU, 아이폰에 IOS 타사 개방 요구 (news.naver.com)",3,https://news.naver.com/mnews/article/029/00029...,"EU, 아이폰에 `iOS` 타사 개방 요구\n본문 바로가기\nNAVER\n뉴스\n연..."
2,"GN⁺: 닌텐도, Palworld 개발사 PocketPair 상대로 특허권 침해 소...",1,https://www.nintendo.co.jp/corporate/release/e...,"News Release : Sep. 19, 2024 ""Filing Lawsuit f..."
3,GN⁺: Scramble - Grammarly 대체 오픈소스 (github.com...,7,https://github.com/zlwaterfield/scramble,GitHub - zlwaterfield/scramble: Open-Source Gr...
4,"Oracle, 이제 JavaScript를 놓아줘야 할 때입니다. (javascrip...",14,https://javascript.tm/,JavaScript™JavaScript™Deliver to: Oracle Corpo...
5,Safari 18.0의 WebKit 기능들 (webkit.org),4,https://webkit.org/blog/15865/webkit-features-...,WebKit Features in Safari 18.0 | WebKit\nWebKi...
6,GN⁺: 책을 쓰지 말아야 하는 이유 (gwern.net),12,https://gwern.net/book-writing,Why To Not Write A Book · Gwern.net\nSkip to m...
7,GN⁺: MrBeast의 프로덕션 성공 비법 (유출된 PDF) (simonwilli...,34,https://simonwillison.net/2024/Sep/15/how-to-s...,How to succeed in MrBeast production (leaked P...
8,The HTTP Query Method (ietf.org),11,https://www.ietf.org/archive/id/draft-ietf-htt...,The HTTP QUERY Method\nInternet-Draft\nThe HTT...
9,"애플, iOS 18 출시 (apple.com)",4,https://www.apple.com/kr/newsroom/2024/09/ios-...,지금껏 가장 개성 넘치며 유능한 iPhone을 위한 iOS 18 출시 - Apple...
